##### Підготовка та Аналіз даних

## Лабораторна робота №2  
## Наука про дані: підготовчий етап

**Мета роботи:**  
Ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.

**Основні поняття:**  
- Сирі дані (raw data)  
- Підготовка даних (data preparation)

---

**ФБ-35**  
**Рабінов Костянтин Костянтинович**


### Імпорт бібліотек та базові налаштування


In [6]:
from datetime import datetime
import urllib.request
import os
import pandas as pd

print("Бібліотеки імпортовано успішно!")


Бібліотеки імпортовано успішно!


### Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних;

In [41]:
def download_vhi_data(region_id):
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/"
        f"get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1981&year2=2020&type=Mean"
    )
    folder = "VHI_Files"
    os.makedirs(folder, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(folder, f"vhi_{region_id}_{timestamp}.csv")
    
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()
        with open(filename, "wb") as file:
            file.write(data)
        print(f"[OK] Дані для регіону {region_id} збережено у файл {filename}")
    except Exception as e:
        print(f"[ERROR]  {region_id}: {e}")
for i in range(1, 28):
    download_vhi_data(i)

[OK] Дані для регіону 1 збережено у файл VHI_Files\vhi_1_2025-03-03_23-09-53.csv
[OK] Дані для регіону 2 збережено у файл VHI_Files\vhi_2_2025-03-03_23-09-56.csv
[OK] Дані для регіону 3 збережено у файл VHI_Files\vhi_3_2025-03-03_23-09-57.csv
[OK] Дані для регіону 4 збережено у файл VHI_Files\vhi_4_2025-03-03_23-09-58.csv
[OK] Дані для регіону 5 збережено у файл VHI_Files\vhi_5_2025-03-03_23-09-58.csv
[OK] Дані для регіону 6 збережено у файл VHI_Files\vhi_6_2025-03-03_23-09-59.csv
[OK] Дані для регіону 7 збережено у файл VHI_Files\vhi_7_2025-03-03_23-10-00.csv
[OK] Дані для регіону 8 збережено у файл VHI_Files\vhi_8_2025-03-03_23-10-01.csv
[OK] Дані для регіону 9 збережено у файл VHI_Files\vhi_9_2025-03-03_23-10-02.csv
[OK] Дані для регіону 10 збережено у файл VHI_Files\vhi_10_2025-03-03_23-10-03.csv
[OK] Дані для регіону 11 збережено у файл VHI_Files\vhi_11_2025-03-03_23-10-04.csv
[OK] Дані для регіону 12 збережено у файл VHI_Files\vhi_12_2025-03-03_23-10-05.csv
[OK] Дані для регіону 

### Зчитати завантажені текстові файли у фрейм (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [42]:
def reader(directory="."):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_files = os.listdir(directory)
    dataframe = pd.DataFrame()
    
    for file in all_files:
        path = os.path.join(directory, file)
        try:
            df = pd.read_csv(path, header=1, names=headers)
            df = df[df["VHI"] != -1]
            df["area"] = file.split("_")[1]
            df["Year"] = df["Year"].str.replace("<tt><pre>", "", regex=False)
            df = df[~df['Year'].str.contains('</pre></tt>')]
            df.drop('empty', axis=1, inplace=True)
            dataframe = pd.concat([dataframe, df]).drop_duplicates().reset_index(drop=True)
        except Exception as e:
            print(f"Error reading file {file}: {e}")
    
    return dataframe

df = reader("VHI_Files")
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95   10
1      1982   2.0  0.063  261.53  55.89  38.20  47.04   10
2      1982   3.0  0.063  263.45  57.30  32.69  44.99   10
3      1982   4.0  0.061  265.10  53.96  28.62  41.29   10
4      1982   5.0  0.058  266.42  46.87  28.57  37.72   10
...     ...   ...    ...     ...    ...    ...    ...  ...
53401  2020  48.0  0.159  277.21  66.52  16.25  41.37    9
53402  2020  49.0  0.150  275.06  65.68  19.44  42.55    9
53403  2020  50.0  0.144  274.00  66.17  18.69  42.43    9
53404  2020  51.0  0.138  272.73  66.79  19.59  43.19    9
53405  2020  52.0  0.132  271.44  67.24  21.90  44.57    9

[53406 rows x 8 columns]


### Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові):

In [ ]:
def id_changer(df):
    region_map = {
        1: "Черкаси", 2: "Чернігів", 3: "Чернівці", 4: "Крим",
        5: "Дніпропетровськ", 6: "Донецьк", 7: "Івано Франківськ", 8: "Харків",
        9: "Херсон", 10: "Хмельницький", 11: "Київська область", 12: "Київ",
        13: "Кіровоград", 14: "Луганськ", 15: "Львів", 16: "Миколаїв",
        17: "Одеса", 18: "Полтава", 19: "Рівне", 20: "Севастополь'",
        21: "Суми", 22: "Тернопіль'", 23: "Закарпаття", 24: "Вінниця",
        25: "Волинь", 26: "Запоріжжя", 27: "Житомир"
    }
    
    df["area"] = pd.to_numeric(df["area"], errors="coerce")
    df = df.dropna(subset=["area"])
    df["area"] = df["area"].astype(int)
    df["area"] = df["area"].replace(region_map)

    print("Оновлена таблиця з українськими назвами областей:")
    print(df)
    return df


df = id_changer(df) 


Оновлена таблиця з українськими назвами областей:
       Year  Week    SMN     SMT    VCI    TCI    VHI          area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95  Хмельницький
1      1982   2.0  0.063  261.53  55.89  38.20  47.04  Хмельницький
2      1982   3.0  0.063  263.45  57.30  32.69  44.99  Хмельницький
3      1982   4.0  0.061  265.10  53.96  28.62  41.29  Хмельницький
4      1982   5.0  0.058  266.42  46.87  28.57  37.72  Хмельницький
...     ...   ...    ...     ...    ...    ...    ...           ...
53401  2020  48.0  0.159  277.21  66.52  16.25  41.37        Херсон
53402  2020  49.0  0.150  275.06  65.68  19.44  42.55        Херсон
53403  2020  50.0  0.144  274.00  66.17  18.69  42.43        Херсон
53404  2020  51.0  0.138  272.73  66.79  19.59  43.19        Херсон
53405  2020  52.0  0.132  271.44  67.24  21.90  44.57        Херсон

[53406 rows x 8 columns]


### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік, пошук екстремумів (min та
max);
- Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;
- Ряд VHI за вказаний діапазон років для вказаних областей;
- Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;


In [ ]:
# Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;
def max_vhi(df, area, year):
    
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].max()

def min_vhi(df, area, year):
   
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].min()

def mean_vhi(df, area, year):
  
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].mean()

def median_vhi(df, area, year):
   
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].median()


for area in df["area"].unique():
    v_max = max_vhi(df, area, "2020")
    v_min = min_vhi(df, area, "2020")
    v_mean = mean_vhi(df, area, "2020")
    v_median = median_vhi(df, area, "2020")
    
    print(f"Область: {area}, рік: 2020")
    print(f"  Максимальне VHI: {v_max}")
    print(f"  Мінімальне VHI: {v_min}")
    print(f"  Середнє VHI: {v_mean:.2f}")   
    print(f"  Медіана VHI: {v_median}")
    print("-" * 40)


v_min_kiev_2020 = min_vhi(df, "Київ", "2020")
print("Мінімальний VHI для Києва у 2020 році:", v_min_kiev_2020)


Область: Хмельницький, рік: 2020
  Максимальне VHI: 67.65
  Мінімальне VHI: 37.43
  Середнє VHI: 49.14
  Медіана VHI: 45.615
----------------------------------------
Область: Київська область, рік: 2020
  Максимальне VHI: 56.3
  Мінімальне VHI: 29.27
  Середнє VHI: 41.36
  Медіана VHI: 39.375
----------------------------------------
Область: Київ, рік: 2020
  Максимальне VHI: 60.83
  Мінімальне VHI: 30.71
  Середнє VHI: 42.71
  Медіана VHI: 41.175
----------------------------------------
Область: Кіровоград, рік: 2020
  Максимальне VHI: 63.63
  Мінімальне VHI: 22.73
  Середнє VHI: 42.00
  Медіана VHI: 40.93
----------------------------------------
Область: Луганськ, рік: 2020
  Максимальне VHI: 64.39
  Мінімальне VHI: 21.29
  Середнє VHI: 40.85
  Медіана VHI: 41.16
----------------------------------------
Область: Львів, рік: 2020
  Максимальне VHI: 63.06
  Мінімальне VHI: 41.35
  Середнє VHI: 50.63
  Медіана VHI: 48.92
----------------------------------------
Область: Миколаїв, рік: 2

In [62]:
# Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);
def vhi_series_and_extremes(df, oblast, year):
 
    subset = df[(df["area"] == oblast) & (df["Year"] == year)]
    
    vhi_values = subset["VHI"]
    
    vhi_min = vhi_values.min()
    vhi_max = vhi_values.max()
    
    print(f"Ряд VHI для області {oblast} за {year} рік:")
    print(vhi_values)
    print(f"Мінімальне VHI: {vhi_min}")
    print(f"Максимальне VHI: {vhi_max}")
    
    return subset


vhi_series_and_extremes(df, "Вінниця", "2020")


Ряд VHI для області Вінниця за 2020 рік:
31596    40.92
31597    43.19
31598    44.74
31599    45.29
31600    44.80
31601    43.92
31602    43.10
31603    42.88
31604    43.71
31605    45.61
31606    44.54
31607    45.23
31608    43.15
31609    40.22
31610    39.12
31611    39.72
31612    40.14
31613    41.91
31614    46.08
31615    50.08
31616    52.05
31617    52.50
31618    54.00
31619    57.18
31620    60.15
31621    61.72
31622    63.51
31623    64.12
31624    62.69
31625    59.11
31626    55.17
31627    51.69
31628    47.52
31629    43.56
31630    40.30
31631    38.84
31632    38.74
31633    36.78
31634    34.73
31635    34.55
31636    36.00
31637    35.34
31638    34.58
31639    34.48
31640    38.34
31641    39.89
31642    43.88
31643    48.01
31644    51.63
31645    50.73
31646    48.70
31647    48.56
Name: VHI, dtype: float64
Мінімальне VHI: 34.48
Максимальне VHI: 64.12


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
31596,2020,1.0,0.078,268.15,70.84,11.00,40.92,Вінниця
31597,2020,2.0,0.077,267.04,69.56,16.82,43.19,Вінниця
31598,2020,3.0,0.077,266.44,68.65,20.84,44.74,Вінниця
31599,2020,4.0,0.078,266.06,66.17,24.40,45.29,Вінниця
31600,2020,5.0,0.080,266.99,63.60,25.99,44.80,Вінниця
31601,2020,6.0,0.085,268.79,61.59,26.25,43.92,Вінниця
31602,2020,7.0,0.092,270.79,61.17,25.04,43.10,Вінниця
31603,2020,8.0,0.106,274.09,65.57,20.20,42.88,Вінниця
31604,2020,9.0,0.126,277.57,71.53,15.88,43.71,Вінниця
31605,2020,10.0,0.150,280.23,76.69,14.54,45.61,Вінниця


In [ ]:
# Ряд VHI за вказаний діапазон років для вказаних областей;
def vhi_series_for_regions_year_range(df, regions, start_year, end_year):
   
    
    subset = df[(df["area"].isin(regions)) &
                (df["Year"] >= start_year) &
                (df["Year"] <= end_year)]
    
   
    print(f"Ряд VHI для областей {regions} за період {start_year} - {end_year}:")
    print(subset[["Year", "Week", "area", "VHI"]])
    
    return subset


vhi_series_for_regions_year_range(df, ["Хмельницький", "Херсон"], 1982, 2020)



Ряд VHI для областей ['Хмельницький', 'Херсон'] за період 1982 - 2020:
       Year  Week          area    VHI
0      1982   1.0  Хмельницький  49.95
1      1982   2.0  Хмельницький  47.04
2      1982   3.0  Хмельницький  44.99
3      1982   4.0  Хмельницький  41.29
4      1982   5.0  Хмельницький  37.72
...     ...   ...           ...    ...
53401  2020  48.0        Херсон  41.37
53402  2020  49.0        Херсон  42.55
53403  2020  50.0        Херсон  42.43
53404  2020  51.0        Херсон  43.19
53405  2020  52.0        Херсон  44.57

[3956 rows x 4 columns]


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,Хмельницький
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,Хмельницький
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,Хмельницький
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,Хмельницький
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,Хмельницький
...,...,...,...,...,...,...,...,...
53401,2020,48.0,0.159,277.21,66.52,16.25,41.37,Херсон
53402,2020,49.0,0.150,275.06,65.68,19.44,42.55,Херсон
53403,2020,50.0,0.144,274.00,66.17,18.69,42.43,Херсон
53404,2020,51.0,0.138,272.73,66.79,19.59,43.19,Херсон


In [95]:
def find_extreme_drought_years(df, threshold=15):
    """
    Шукає роки, коли VHI < threshold у >=5 областях (що становить 20% від 25).
    """
    # Кількість областей, яку вважаємо "понад 20% з 25"
    required_count = 5
    
    # Групуємо за (Year, area) і беремо мінімальне значення VHI за рік
    grouped = df.groupby(["Year", "area"], as_index=False)["VHI"].min().rename(columns={"VHI": "min_VHI"})
    
    drought_years = {}
    for year in sorted(grouped["Year"].unique()):
        year_data = grouped[grouped["Year"] == year]
        # Фільтруємо області, де мінімальний VHI < threshold
        affected = year_data[year_data["min_VHI"] < threshold]
        if len(affected) >= required_count:
            drought_years[year] = list(zip(affected["area"], affected["min_VHI"]))
    
    if drought_years:
        print(f"Роки, коли VHI < {threshold} у >=5 областях (20% від 25):")
        for yr, info in drought_years.items():
            print(f"  Рік {yr}:")
            for (area_name, min_vhi) in info:
                print(f"    {area_name} – мін. VHI: {min_vhi}")
    else:
        print(f"Не знайдено років, коли VHI < {threshold} торкнувся >=5 областей.")
    
    return drought_years
find_extreme_drought_years(df, threshold=15)

Роки, коли VHI < 15 у >=5 областях (20% від 25):
  Рік 2000:
    Вінниця – мін. VHI: 11.25
    Київ – мін. VHI: 6.49
    Київська область – мін. VHI: 10.6
    Севастополь' – мін. VHI: 8.14
    Харків – мін. VHI: 9.36
    Черкаси – мін. VHI: 10.68
  Рік 2007:
    Запоріжжя – мін. VHI: 10.88
    Крим – мін. VHI: 13.28
    Миколаїв – мін. VHI: 5.94
    Одеса – мін. VHI: 5.52
    Херсон – мін. VHI: 12.23


{2000: [('Вінниця', 11.25),
  ('Київ', 6.49),
  ('Київська область', 10.6),
  ("Севастополь'", 8.14),
  ('Харків', 9.36),
  ('Черкаси', 10.68)],
 2007: [('Запоріжжя', 10.88),
  ('Крим', 13.28),
  ('Миколаїв', 5.94),
  ('Одеса', 5.52),
  ('Херсон', 12.23)]}